# Requirements

In [53]:
%%capture
!pip install pdfplumber
import pdfplumber
import requests
import os
import pandas as pd


# Funciones

## Obtener odm provisorio
(Solo necesito las columnas DNI, Nombre, Apellido. Para que a partir del nombre habia hecho el join con ns_def.csv donde le asigno el sexo segun el nombre. Esto es asi porque luego en el orden definitivo, pusieron Apellido y Nombre junto, siendo dificil y quizas imposible separar los nombres compuestos y apellidos compuestos para poder obtener solo los nombres y asi estimar si es Femenino o Masculino)

In [54]:
def obtener_odm(url):
  dfodm = pd.read_csv(url)
  dfodm.columns = dfodm.columns.str.replace("\n", " ", regex=True)
    # Renombrar columnas
  rename_dict = {
        "Número de documento": "DNI",
        "Apellido": "APELLIDO",
        "Nombre": "NOMBRE" }
  dfodm = dfodm.rename(columns=rename_dict)
  columnas_a_eliminar = ['PROMEDIO_CARRERA', 'FECHA_TITULO', 'ESPECIALIDAD', 'NOTA_EXAMEN', 'TIPO_UNI_x',
                      'COMPONENTE', 'PUNTAJE', 'ODM', 'PUNTAJE_CRUDO', 'DIAS_DESDE_TITULO', 'ORIGEN',
                      'TIPO_UNI_y', 'PAIS_UNI', 'UNI','SEXO']

  dfodm.drop(columns=columnas_a_eliminar, inplace=True)
  dfodm['DNI'] = dfodm['DNI'].astype(str)
  return dfodm
# como no encuentro el odm provisiorio original uso este ya estilizado pero para tomar lo de los nombres nomas,
# quizas me joda el tema columna sexo.. la elimino

## Obtener ODM definitivo

In [55]:
def obtener_ODM2025(url_pdf: str, nombre_archivo: str = "ODM2025.pdf") -> pd.DataFrame:
    """
    Descarga, extrae y limpia datos de un PDF en un DataFrame listo para análisis.
    Args: url_pdf (str): URL del archivo PDF a descargar.
          nombre_archivo (str): Nombre con que se guardará el PDF localmente.
    Returns:  pd.DataFrame: DataFrame
    """
    # Descargar PDF si no existe localmente
    if not os.path.isfile(nombre_archivo):
        r = requests.get(url_pdf)
        if r.status_code == 200:
            with open(nombre_archivo, "wb") as f:
                f.write(r.content)
        else:
            print(f"Error downloading PDF from {url_pdf}. Status code: {r.status_code}")
            return None

    # Extraer tablas del PDF
    data = []
    try:
        with pdfplumber.open(nombre_archivo) as pdf:
            for page in pdf.pages:
                table = page.extract_table()
                if table:
                    data.extend(table)
    except Exception as e:
        print(f"Error opening or reading PDF file {nombre_archivo}: {e}")
        return None

    dfODM2025 = pd.DataFrame(data[1:], columns=data[1])
    dfODM2025.columns = dfODM2025.columns.str.replace("\n", " ", regex=True)
    columnas_a_eliminar = ['Apellido y Nombre']
    dfODM2025.drop(columns=columnas_a_eliminar, inplace=True)

    return dfODM2025

## Join odm (Nombre, Apellido) con ODM, a partir del DNI

In [56]:
def mergeODFS (df1, df2):
  # Supongamos que ya tienes los DataFrames:
  # df1 con columnas: DNI, NOMBRE, APELLIDO (dfodm)
  # df2 con columnas incluyendo DNI, NOMBRE, APELLIDO, PUNTAJE_CRUDO, ODM_CRUDO, PROMEDIO, ESPECIALIDAD (dfODM2025)

  # Hacer merge con df1 para traer NOMBRE y APELLIDO
  df = df2.merge(df1[['DNI', 'NOMBRE', 'APELLIDO']], on='DNI', how='left')

  return df

## Limpiar el df

In [65]:
def limpiar_df(df):

    # Renombrar columnas
    rename_dict = {
        "Institución formadora": "UNIVERSIDAD",
        "Promedio": "PROMEDIO_CARRERA",
        "Apellido": "APELLIDO",
        "Nombre": "NOMBRE",
        "Fecha de Expedición de Título": "FECHA_TITULO",
        "Especialidad": "ESPECIALIDAD",
        "Puntaje obtenido en el examen": "NOTA_EXAMEN",
        "Tipo Uni": "TIPO_UNI",
        "Componente": "COMPONENTE",
        "Puntaje Final": "PUNTAJE"
    }
    df = df.rename(columns=rename_dict)

    # Eliminar filas no deseadas
    df = df[~df.isin(["DNI"]).any(axis=1)] # filas donde se repite el encabezado
    df = df[df['DNI'].str.strip().astype(bool)]  # elimina filas vacías (con la celda de apellido vacio)
    df = df.dropna().reset_index(drop=True) # reindexar


    # Ajustes de formato
    df["FECHA_TITULO"] = pd.to_datetime(df["FECHA_TITULO"], format="%d-%m-%Y", errors="coerce")

    # Floats
    cols_f = ["PROMEDIO_CARRERA", "PUNTAJE"]
    df[cols_f] = df[cols_f].replace(",", ".", regex=True).replace("", float('nan')).astype(float)
    # Enteros
    cols_i = ['NOTA_EXAMEN', 'COMPONENTE', 'ODM']
    for col in cols_i:
        df[col] = pd.to_numeric(df[col], errors="coerce").astype("Int64")

    df["COMPONENTE"] = df["COMPONENTE"].fillna(0)

    df['PUNTAJE_CRUDO'] = df['PUNTAJE']-df['COMPONENTE']

    # Reemplazos globales en todas las celdas
    df = df.replace({r'\n': ' ', "En trámite": "30-06-2025"}, regex=True) #crear el espacio entre los nombres en vez de "\n" y poner la fecha que elegi 30 junio 2025 en vez de "en tramite"
    # Tiempo entre recibido y el examen (1 julio 2025)
    # Definir la fecha cero
    fecha_cero = pd.to_datetime("2025-07-01")

    # Asegurar que FECHA_TITULO es datetime
    df["FECHA_TITULO"] = pd.to_datetime(df["FECHA_TITULO"], format="%d-%m-%Y", errors="coerce")
    # Calcular diferencia en días
    df["DIAS_DESDE_TITULO"] = (fecha_cero - df["FECHA_TITULO"]).dt.days

    return df

## Mapeo de SEXO segun NOMBRE

In [58]:
def mapear_sexo_por_primer_nombre(df, url, nombre_col_original='NOMBRE', sexo_col='SEXO'):

    # Descargar y leer el archivo CSV si no existe localmente
    # Nombre del archivo local
    file_name = url.split("/")[-1]

    if not os.path.isfile(file_name):
        r = requests.get(url)
        with open(file_name, "wb") as f:
            f.write(r.content)

    ns_def = pd.read_csv(file_name)

    # Renombrar columna del archivo descargado para homogeneizar
    rename_dict = {"primer_nombre": "NOMBRE"}
    ns_def = ns_def.rename(columns=rename_dict)

    # Extraer primer nombre, limpiar y pasar a mayúscula
    df['primer_nombre'] = df[nombre_col_original].apply(lambda x: x.split()[0] if isinstance(x, str) else "")
    df['primer_nombre'] = df['primer_nombre'].str.strip().str.upper()
    ns_def['NOMBRE'] = ns_def['NOMBRE'].str.strip().str.upper()

    # Crear diccionario para mapeo de sexo
    dic_sexo = dict(zip(ns_def['NOMBRE'], ns_def[sexo_col]))

    # Mapear sexo usando el primer nombre
    df['SEXO'] = df['primer_nombre'].map(dic_sexo)

    # Marcar como 'ND' los casos sin coincidencia
    df['SEXO'] = df['SEXO'].fillna('ND')

    # Eliminar columna auxiliar
    df.drop(columns=['primer_nombre'], inplace=True)

    return df

## ORIGEN de postulante segun DNI >/<50millones

In [59]:
def asignar_origen(df, columna_dni='DNI'):
    # Crear columna ORIGEN según condición del DNI
    df['DNI'] = pd.to_numeric(df['DNI'], errors='coerce').astype('Int64')
    df['ORIGEN'] = df[columna_dni].apply(lambda x: 'arg' if x < 50000000 else 'extr')

    return df

## ODM sin 5 puntos de TIPO_UNI (COMPONENTE)

In [60]:
def asignar_ODM_crudo(df):
    df = df.sort_values(
        by=['ESPECIALIDAD', 'PUNTAJE_CRUDO', 'NOTA_EXAMEN', 'PROMEDIO_CARRERA', 'DNI'],
        ascending=[True, False, False, False, True]
    )
    df['ODM_CRUDO'] = df.groupby('ESPECIALIDAD').cumcount() + 1
    return df

## Mapeo de UNIVERSIDADES
(En un primer momento identifique en el odm provisorio que los nombres de UNIVERSIDADES no estaban normalizado y tambien que habia hosptales entre esos nombres: hospitales argentinos sin universidad de medicina. A estos los asigne a la UBA.)
(PENDIENTE: chequear en ODM definitivo si hay estos errores en las universidades)

In [61]:
def mapear_universidades(df, file_name, nombre_col_original='UNIVERSIDAD'):

    # Descargar y leer el archivo CSV si no existe localmente
    # Nombre del archivo local
    """
    file_name = url.split("/")[-1]

    "if not os.path.isfile(file_name):
        r = requests.get(url)
        with open(file_name, "wb") as f:
            f.write(r.content)

    """
    universidades = pd.read_csv(file_name)

    # Hacer merge con el df original usando la columna UNIVERSIDAD como clave
    df = df.merge(universidades[['UNIVERSIDAD','UNI', 'CLASE_UNI', 'PAIS_UNI','CIUDAD_UNI']],
                         left_on = nombre_col_original, right_on = 'UNIVERSIDAD', how='left')

    # Eliminar la columna original de universidad
    df = df.drop(columns=[nombre_col_original])

    return df

# Main

In [66]:
urlodm = "https://raw.githubusercontent.com/LuisaBeccar/ODMexamen/refs/heads/main/odm_2.0.csv"
dfodm = obtener_odm(urlodm)

urlPDFODM2025gh = "https://raw.githubusercontent.com/LuisaBeccar/ODMexamen/main/generar_data/ODM2025.pdf"
dfODM2025 = obtener_ODM2025(urlPDFODM2025gh)

df = mergeODFS(dfodm, dfODM2025)
df = limpiar_df(df)

urlsexo = "https://raw.githubusercontent.com/LuisaBeccar/ODMexamen/main/generar_data/ns_def.csv"
df = mapear_sexo_por_primer_nombre(df, urlsexo, nombre_col_original='NOMBRE', sexo_col='SEXO')

df = asignar_origen(df, columna_dni='DNI')

df = asignar_ODM_crudo(df)

urluni = "https://raw.githubusercontent.com/LuisaBeccar/ODMexamen/main/generar_data/universidades.csv"
df = mapear_universidades(df, urluni, nombre_col_original='UNIVERSIDAD')


In [67]:
df.columns

Index(['DNI', 'ESPECIALIDAD', 'PROMEDIO_CARRERA', 'FECHA_TITULO',
       'NOTA_EXAMEN', 'TIPO_UNI', 'COMPONENTE', 'PUNTAJE', 'ODM', 'NOMBRE',
       'APELLIDO', 'PUNTAJE_CRUDO', 'DIAS_DESDE_TITULO', 'SEXO', 'ORIGEN',
       'ODM_CRUDO', 'UNI', 'CLASE_UNI', 'PAIS_UNI', 'CIUDAD_UNI'],
      dtype='object')

In [69]:
nuevo_orden = ['DNI', 'NOMBRE', 'APELLIDO', 'SEXO', 'ORIGEN',
               'UNI','TIPO_UNI', 'PAIS_UNI', 'CIUDAD_UNI', 'CLASE_UNI',
               'FECHA_TITULO', 'DIAS_DESDE_TITULO', 'PROMEDIO_CARRERA', 'ESPECIALIDAD',
               'NOTA_EXAMEN', 'COMPONENTE', 'PUNTAJE', 'PUNTAJE_CRUDO', 'ODM', 'ODM_CRUDO']

df = df[nuevo_orden]
df

,DNI,NOMBRE,APELLIDO,SEXO,ORIGEN,UNI,TIPO_UNI,PAIS_UNI,CIUDAD_UNI,CLASE_UNI,FECHA_TITULO,DIAS_DESDE_TITULO,PROMEDIO_CARRERA,ESPECIALIDAD,NOTA_EXAMEN,COMPONENTE,PUNTAJE,PUNTAJE_CRUDO,ODM,ODM_CRUDO
0,28494730,PATRICIO LEANDRO,ACOSTA,M,arg,UNIVERSIDAD DE BUENOS AIRES,N,Argentina,Buenos Aires,Pública,2022-11-08,966.0,8.14,Alergia e inmunología,83,5,54.64,49.64,1,1
1,96354434,VANESSA PAOLA,GUTIERREZ ECHEVERRY,F,extr,UNIVERSIDAD DEL NORTE,E,Paraguay,Asunción,Privada,2022-12-16,928.0,8.68,Alergia e inmunología,72,0,44.68,44.68,2,2
2,42490362,JOAQUIN,ROSATO,M,arg,UNIVERSIDAD DE BUENOS AIRES,N,Argentina,Buenos Aires,Pública,2025-03-28,95.0,7.35,Anatomía patológica,86,5,55.35,50.35,1,1
3,42375593,PILAR MARIA,BERETERBIDE,F,arg,UNIVERSIDAD DE BUENOS AIRES,N,Argentina,Buenos Aires,Pública,2025-03-27,96.0,7.50,Anatomía patológica,84,5,54.50,49.5,2,2
4,41398382,FRANCISCO AGUSTIN,MARTELLA,M,arg,UNIVERSIDAD DE BUENOS AIRES,N,Argentina,Buenos Aires,Pública,2024-12-18,195.0,8.88,Anatomía patológica,79,5,53.38,48.38,3,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6122,96317237,ITUKA ELENA,ELJADUE BLANCO,F,extr,UNIVERSIDAD DE CARTAGENA,E,Colombia,Cartagena,Pública,2022-06-23,1104.0,7.78,Urología,45,0,30.28,30.28,148,147
6123,96264902,ANDRES GABRIEL,SORIA FREIRE,M,extr,UNIVERSIDAD NACIONAL DE CHIMBORAZO,E,Ecuador,Riobamba,Pública,2021-11-25,1314.0,8.76,Urología,41,0,29.26,29.26,149,148
6124,37889490,BRUNO ABEL,VILLALBA,M,arg,UNIVERSIDAD NACIONAL DEL CHACO AUSTRAL,N,Argentina,Chaco,Pública,2024-03-07,481.0,6.06,Urología,44,5,33.06,28.06,138,149
6125,96310398,ESNEIDER DAVID,ESCOBAR VALENCIA,M,extr,UNIVERSIDAD METROPOLITANA DE BARRANQUILLA,E,Colombia,Barranquilla,Privada,2017-07-28,2895.0,7.36,Urología,41,0,27.86,27.86,150,150


# Descarga BaseDatos

In [85]:
df.to_csv('BaseODM2025.csv', index=False)
from google.colab import files
files.download('BaseODM2025.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Analisis

In [71]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6127 entries, 0 to 6126
Data columns (total 20 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   DNI                6127 non-null   Int64         
 1   NOMBRE             6127 non-null   object        
 2   APELLIDO           6127 non-null   object        
 3   SEXO               6127 non-null   object        
 4   ORIGEN             6127 non-null   object        
 5   UNI                5415 non-null   object        
 6   TIPO_UNI           6127 non-null   object        
 7   PAIS_UNI           5415 non-null   object        
 8   CIUDAD_UNI         5380 non-null   object        
 9   CLASE_UNI          5380 non-null   object        
 10  FECHA_TITULO       4821 non-null   datetime64[ns]
 11  DIAS_DESDE_TITULO  4821 non-null   float64       
 12  PROMEDIO_CARRERA   6127 non-null   float64       
 13  ESPECIALIDAD       6127 non-null   object        
 14  NOTA_EXA

In [72]:
df.describe(include=object)

,NOMBRE,APELLIDO,SEXO,ORIGEN,UNI,TIPO_UNI,PAIS_UNI,CIUDAD_UNI,CLASE_UNI,ESPECIALIDAD
count,6127,6127,6127,6127,5415,6127,5415,5380,5380,6127
unique,4144,5447,2,2,133,2,15,73,2,41
top,CAMILA,GONZALEZ,F,arg,UNIVERSIDAD DE BUENOS AIRES,N,Argentina,Buenos Aires,Pública,Cirugía general
freq,54,26,3815,3626,2017,4054,3506,2368,3952,669


In [73]:
df['ESPECIALIDAD'].value_counts()

,count
ESPECIALIDAD,
Cirugía general,669
Anestesiología,591
Tocoginecología,476
Pediatría y pediátricas articuladas,470
Ortopedia y traumatología,464
Diagnóstico por imágenes,373
Cardiología,358
Dermatología,330
Oftalmología,321


In [84]:
df[df['PAIS_UNI']=='Argentina']['UNI'].value_counts()

,count
UNI,
UNIVERSIDAD DE BUENOS AIRES,2017
UNIVERSIDAD NACIONAL DE LA PLATA,328
UNIVERSIDAD NACIONAL DEL NORDESTE,138
UNIVERSIDAD NACIONAL DE MAR DEL PLATA,100
UNIVERSIDAD NACIONAL DE LA MATANZA,88
UAI,85
UNIVERSIDAD NACIONAL DE TUCUMAN,83
UNIVERSIDAD NACIONAL DE ROSARIO,82
UNIVERSIDAD NACIONAL DEL LITORAL,65


In [75]:
df[['PROMEDIO_CARRERA', 'NOTA_EXAMEN', 'PUNTAJE','PUNTAJE_CRUDO']].describe()

,PROMEDIO_CARRERA,NOTA_EXAMEN,PUNTAJE,PUNTAJE_CRUDO
count,6127.000000,6127.0,6127.000000,6127.0
mean,7.467243,67.849355,44.700228,41.391921
std,1.199084,10.815431,7.147596,5.823636
min,0.000000,28.0,14.500000,14.5
25%,6.830000,61.0,40.195000,37.87
50%,7.600000,70.0,45.950000,42.19
75%,8.285000,76.0,50.085000,45.6
max,9.860000,93.0,60.570000,55.57
